In [ ]:
import warnings, csv, os, sys, json
import pandas as pd
import json_repair
from typing import List
warnings.filterwarnings("ignore")
from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [ ]:
def write_csv(output_file, data):
    with open(output_file, 'a', newline='', encoding='utf-8') as outfile:  
        writer = csv.writer(outfile)
        writer.writerow(data)

In [ ]:
def write_txt(txt_path, data):
    with open(txt_path, 'a') as txf:
        txf.write(data)

In [ ]:
#variables

model_name = 'gpt-4-1106-preview'  #gpt model name
audience = ' US owners of 1 dog'  #audience
output_file = 'run2.csv'
txt_file = 'output.txt'

#you can change questions and traits/count as you want, this part has become dynamic in this notebook.
questions = [
"How old is your dog (in # of years)?",
"What is your dog's primary breed?",
"Approximately how much does your dog weigh (in pounds)?",
"What is your dog's name?",
"What was the inspiration for that name?",
"What is the most common nickname (if any) that you use for your dog?",
"When did you get your dog? (Before 2020, In 2020, After 2020)",
"Did you adopt your dog? (yes/no)",
"How old was your dog when you got him/her? Provide estimated number of months old.",
"How did you feel the first time you met your dog?",
"How has your dog changed your outlook on life?",
"On a scale of 1-5, where 1 is not at all important and 5 is very important, how important is your dog's happiness to your overall well-being?",
"Choose the term that best describes your dog's relationship with you.(Best Friend,Loyal companion,Family Member,Protector,None of these apply)",
"Why did you choose that term?",
"What is the most recent command or trick your dog has learned, and how long did it take to train?",
"How many hours a day does your dog typically sleep and where does your dog usually sleep in your home?",
"Have you shopped for your dog online in the past year? (yes/no)",
"Over the past year, approximately how much did you spend on your dog? Provide total $ amount.",
"Over the past year, approximately what percent of the money you spent on your dog was paid to your veterinarian? Choose a percent from 0% to 100%.",
"Over the past year, approximately what percent of the money you spent on your dog was on dog food? Choose a percent from 0% to 100%.",
"In March, approximately much money did you spend on your dog? Provide estimated total $ amount.",
"How did the amount of money you spent on your dog in March compare with February?(Spent about the same in March,I'm not sure,Spent more in March,Spent less in March)",
"How often do you purchase dog food? (Bi-weekly,Monthly,Less frequently than monthly,Weekly)",
"Do you primarily feed your dog dry food, wet food, or a mix?",
"Which of the following statements best describes your level of familiarity of your dog's primary food brand?(I know a lot about the brand,I know just the basics about the brand,I'm not very familiar with the brand)",
"On a scale of 1-5, where 1 is not at all important and 5 is very important, how important is the following in choosing your primary dog food brand? Quality of ingredients",
"Price",
"Nutritional value of the dog food",
"Brand reputation",
"How your dog likes the taste",
"Your veterinarian's recommendation",
"Special dietary needs",
"What dog food brand do you give your dog the most?",
"What are the main 2-3 reasons you choose this brand?",
"On a scale of 1-5, where 1 is not at all satisfied and 5 is very satisfied, how satisfied are you with the following for your primary dog food brand? Quality of ingredients",
"Price",
"Nutritional value of the dog food",
"Brand reputation",
"How your dog likes the taste",
"Your veterinarian's recommendation",
"Special dietary needs",
"How many times a day does your dog typically go out for a walk?(0,1,2,3,4,5,more than 5,not sure)",
"How (if at all) have recent events affected how you interact with your dog?(No change,Not sure,I interact more,I interact less)",
"What events are affecting your interaction level most?",
"Are you planning to watch the Summer Olympics with your dog?(No,Yes,Not Sure)",
"Are you planning to travel this Spring with your dog?(No,Yes,Not Sure)",
"If planning to travel this Spring with your dog, which is your preferred method of transportation? Select the best option.(Bus,Car,N/A-I'm not planning to travel with my dog this Spring,Other)",
"What is your gender?",
"What is your age?",
"How many people, including yourself, live in your household?(1,2,3,5 or more)",
"What is the highest level of education you have completed?(Associate degree, Bachelor's degree,Graduate degree, High school diploma or equivalent, Some college, no degree)",
"What is your current employment status?(Employed full-time,Employed part-time,Retired,Self-employed,Unable to work,Unemployed)",
"What is your total annual household income before taxes?($100,000 to $149,000,$150,000 or more,$25,000 to $49,999,$50,000 to $74,999, Less than $25,000, Prefer not to say)",
"In what city do you currently reside?",
"In what state do you currently reside?"
]

traits_and_counts = [
    ('Men aged 60+', 19)
]


In [ ]:
# This block of code is not needed, it is there to know how function calling works.

# To learn more about function calling, check this out https://www.datacamp.com/tutorial/open-ai-function-calling-tutorial
#Answer is the class which contains response given by each user, it also contain traits to know which person answered the questions.
#llm is inteligent enough to give unique response each time based on the given system and user prompt, we should not worry about it.
class Answer(BaseModel):
    """Human-like set of answers told by each person while being surveyed. 
    Each person should have their own unique set of answers that should be different from other person's set of answers."""

    answer1: str = Field(description="This is the human like answer to Question1.")
    answer2: str = Field(description="This is the human like answer to Question2.")
    answer3: str = Field(description="This is the human like answer to Question3.")
    answer4: str = Field(description="This is the human like answer to Question4.")
    traits: str = Field(description="This is the trait of the person.")
    
    
    
#list of answers, the formatted list that the llm model will generate which contains dictionaries of Answer class
#llm model will make sure that each dictionary in the list is unique from each other as it will treat the list like list of human responses.
class Answers(BaseModel):
    """Human-like survey answers to tell user."""

    answer: List[Answer]

In [ ]:
# This is standard pattern of list of functions used for function calling.
# To know the pattern, you can print out openai_functions variable in previous New.ipynb file
# This function is responsible to give the output list of answers of each human that we want, based on the parameters provided.
# To learn more about function calling, check this out https://www.datacamp.com/tutorial/open-ai-function-calling-tutorial
def create_openai_functions(questions: List):
    fn  = [{'name': 'Answers',
            'description': 'Human-like survey answers to tell user.',
            'parameters': {'type': 'object',
            'properties': {'answer': {'type': 'array',
                'items': {'description': "Human-like set of answers told by each person while being surveyed. \nEach person should have their own unique set of answers that should be different from other person's set of answers.",
                'type': 'object',
                'properties': {
                'traits': {'description': 'This is the trait of the person.',
                    'type': 'string'}},
                'required': ['traits']}}},
            'required': ['answer']}}]
    answers = fn[0]['parameters']['properties']['answer']['items']['properties']
    required = fn[0]['parameters']['properties']['answer']['items']['required']
    for ind, _ in enumerate(questions, 1):
        answers[f'answer{ind}'] = {'description': f'This is the human like answer to Question{ind}.', 'type': 'string'}
        required.append(f'answer{ind}')
    return fn
        

In [ ]:
#sample openai function example
openai_functions = create_openai_functions(questions)

In [ ]:
parser = JsonKeyOutputFunctionsParser(key_name="answer")  # to parse the output, it will return list of answers based on function calling

In [ ]:
#bind openai_functions to llm for function calling
llm = ChatOpenAI(
    temperature=0.9,
    model_name=model_name,
    max_tokens=4095
).bind(functions=openai_functions)

In [ ]:
#this is default system message, you can change it based on your need.
system_message = """
You are a survey answering bot that generates answers like a survey when questions are asked. The answer should be made as if you are a human. 
Give answers assuming you are a new human with different lifestyles while giving answers.
Give answers with new thoughts, new ideas, new moods, you can also choose to answer very rudely, but the main idea is to be random, do not try to repeat same answers since humans have different answers with different answer style.
While giving answers, you should be as creative as possible and you should deviate your answers as much as possible from previous answers.
In every answer, change styles of answers, change average sentence lengths of answer, change fk_grade_level of sentences of answer. Make it different from previous answers. But also make sure it is the answer given by a human. So, don't make it seem like it is AI generated. Add both simple and fancy words.
In 1 answer, give your answers assuming you are having a worse life, in another answer, give your answers assuming you are having best life. like this, keep on changing the lifestyle of human that you are.
Avoid same repeated answers as much as possible.
Do no repeat same pattern in each answers. Give short answers sometimes and sometimes long answers, be random.
Since, human can give both positive and negative answers, you should follow the same principles.
Your answers should be descriptive just like human answers.
Each set of answers should be different from another set of answers. 
If you are asked about 'top few things' or 'few things', each answers should have random number of comma separated sentences. For example:
sentence1, sentence2, and sentence3. (3 sentences)
sentence1. (1 sentence)
sentence1, sentence2, sentence3, sentence4, and sentence5. (5 sentences)
sentence1, sentence2. (2 sentences)
sentence1, sentence2, sentence3, and sentence4. (4 sentences)


If the output asks for a monetary output give just the dollar amount and no text before or after it unless prompted to do so. So for example if the answer is $40 just give $40.
If the output asks for a decimal output give just the value and no text before or after it unless prompted to do so. So for example if the answer is 40.5 just give 40.5.
If you are asked a question like where do you live? Be sure to just answer the place that you live. No text before or after it is needed unless prompted to do so.
If the answer is asking for a percentage output just give the value. So if the answer is 45% just give 45%. Do not give any text before or after that unless prompted to do so.
"""

In [ ]:
#chat template
prompt = ChatPromptTemplate.from_messages(
    [("system", system_message), ("user", "{input}")]
)

In [ ]:
#chain using prompt, llm and parser
#it uses the prompt with llm and generate the answer based on the parser

# chain = prompt | llm | parser

chain = prompt | llm 

In [ ]:
def output_parser(response):
    try:
        return eval(response.additional_kwargs['function_call']['arguments'])['answer']
    except Exception as e:
        # print("Error in Eval\n")
        # print(e)
        # print('----------------------------------------------------------------------')
        try:
            return json_repair.loads(response.additional_kwargs['function_call']['arguments'])['answer']
        except Exception as e:
            # print("Error in Json loads")
            # print(e)
            # print('----------------------------------------------------------------------')
            write_csv(txt_file, "\n-Failed-eval-json-loads---------------------------------------\n")
            return None

In [ ]:
total_processed_row = 0  #total row processed
total_remaining_surveyed = 0  #total people remaining to be surveyed
person_number = 0  #person count
traits_dict = {}


for dta in traits_and_counts:
    traits_dict[dta[0]] = int(dta[1])  #creating dict of traits and counts
    total_remaining_surveyed += int(dta[1])  #total people remaining to be surveyed
        
if output_file not in os.listdir():  #if csv is not available
    write_csv(output_file, ['Traits', 'Person Number']+questions)
else: #if csv already exists
    df = pd.read_csv(output_file)
    old_traits_dict = df['Traits'].value_counts().to_dict() #get traits/counts of already surveyed people in csv
    for k,v in old_traits_dict.items():
        person_number += v #increasing the person number
        if k not in traits_dict.keys(): 
            continue
        traits_dict[k] = traits_dict[k]-v
        total_remaining_surveyed = int(total_remaining_surveyed - v)  #subtracting from the people that are already in csv. suppose 7 needed, 3 are already in csv, it will only process 4
        
        
print("Remaining Traits/counts dict:", traits_dict)
print("Total remaining people to be surveyed:", total_remaining_surveyed)
print()

if total_remaining_surveyed <= 0:
    print("All the people are surveyed, if you want to increase survey count, increase the count in traits_and_counts variable at top")
    sys.exit()
    
    
while True:
    total_surveyed = sum([counts for _, counts in traits_dict.items()])
    input_message = f"Generate survey answers from {total_surveyed} people.\nAll of the surveyed people are {audience}\n"
    for traits, counts in traits_dict.items():
        if counts <= 0:
            continue
        input_message += f'{counts} of the surveyed people have this trait: {traits}\n'
    input_message += f'Extremely Important Note: You must compulsory give answers to all the questions provided below. Do not skip any questions.\n'
    for ind, question in enumerate(questions, 1):
        input_message += f'Question{ind}: {questions[ind-1]}\n'
    try:
        res = chain.invoke({"input": input_message})
    except Exception as e:
        print('OpenAI Error', e)
        print()
        print()
        continue
    write_csv(txt_file, res)
    responses = output_parser(res)
    if responses is None:
        continue
    final_responses = []
    for data in responses:
        continue_for = False
        if type(data) == dict:
            try:
                data['traits']
            except:
                write_csv(txt_file, "\n-Traits-not-found---------------------------------------\n")
                continue
            for ind, question in enumerate(questions):
                try:
                    data[f'answer{ind+1}']
                except:
                    write_csv(txt_file, f"\n-answer{ind+1}-not-found---------------------------------------\n")
                    continue_for = True
                    break
            if continue_for:
                continue
            try:
                traits_dict[data['traits']] = traits_dict[data['traits']]-1
            except:
                for kk in traits_dict.keys():
                    if kk in data['traits']:
                        traits_dict[kk] = traits_dict[kk]-1
                        data['traits'] = kk
                        break
                else:
                    write_csv(txt_file, "\n-Traits-not-matched---------------------------------------\n")
                    continue
            final_responses.append(data)
    for data in final_responses:
        person_number += 1
        lst = [data['traits'], person_number]
        for ind, question in enumerate(questions):
            lst.append(data[f'answer{ind+1}'])
        write_csv(output_file, lst)
    write_csv(txt_file, "\n-Completed---------------------------------------\n")
    total_processed_row += len(final_responses)
    print('Remaining traits:', traits_dict)
    print("Total responses given by model:", len(final_responses))
    print("Total processed responses:", total_processed_row)
    print(final_responses)
    print()
    print()
    if total_processed_row >= total_remaining_surveyed:
        break
    

In [ ]:
from langchain_core.messages.ai import AIMessage


In [ ]:
a = AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{""answer"":[{""traits"":""Female aged 60+"",""answer1"":""5"",""answer2"":""Labrador Retriever"",""answer3"":""75"",""answer4"":""Buddy"",""answer5"":""It was my late husband\'s nickname"",""answer6"":""Bud"",""answer7"":""Before 2020"",""answer8"":""yes"",""answer9"":""6"",""answer10"":""Instant connection"",""answer11"":""It\'s taught me more about unconditional love"",""answer12"":""5"",""answer13"":""Family Member"",""answer14"":""Because he\'s part of the family"",""answer15"":""Paw shake, about a week"",""answer16"":""14, in a dog bed by my bedside"",""answer17"":""yes"",""answer18"":""$1200"",""answer19"":""20%"",""answer20"":""60%"",""answer21"":""$100"",""answer22"":""Spent less in March"",""answer23"":""Monthly"",""answer24"":""Dry food"",""answer25"":""I know a lot about the brand"",""answer26"":""5"",""answer27"":""3"",""answer28"":""5"",""answer29"":""4"",""answer30"":""5"",""answer31"":""4"",""answer32"":""2"",""answer33"":""Blue Buffalo"",""answer34"":""High quality, my dog loves it, good value"",""answer35"":""5"",""answer36"":""4"",""answer37"":""5"",""answer38"":""4"",""answer39"":""5"",""answer40"":""4"",""answer41"":""3"",""answer42"":""3"",""answer43"":""I interact more"",""answer44"":""The ongoing pandemic"",""answer45"":""No"",""answer46"":""Yes"",""answer47"":""Car"",""answer48"":""Female"",""answer49"":""63"",""answer50"":""1"",""answer51"":""Graduate degree"",""answer52"":""Retired"",""answer53"":""Less than $25,000"",""answer54"":""Sarasota"",""answer55"":""Florida""},{""traits"":""Female aged 60+"",""answer1"":""3"",""answer2"":""Beagle"",""answer3"":""20"",""answer4"":""Scout"",""answer5"":""To Kill a Mockingbird is my favorite book"",""answer6"":""Scooty"",""answer7"":""After 2020"",""answer8"":""no"",""answer9"":""8"",""answer10"":""Overwhelmed with joy"",""answer11"":""I\'ve become more active"",""answer12"":""5"",""answer13"":""Best Friend"",""answer14"":""We do everything together"",""answer15"":""Stay, a couple of days"",""answer16"":""16, on the living room couch"",""answer17"":""yes"",""answer18"":""$800"",""answer19"":""30%"",""answer20"":""50%"",""answer21"":""$70"",""answer22"":""Spent more in March"",""answer23"":""Bi-weekly"",""answer24"":""Mix"",""answer25"":""I know just the basics about the brand"",""answer26"":""5"",""answer27"":""4"",""answer28"":""5"",""answer29"":""3"",""answer30"":""5"",""answer31"":""2"",""answer32"":""4"",""answer33"":""Purina"",""answer34"":""Affordable, accessible, and Scout enjoys it"",""answer35"":""4"",""answer36"":""5"",""answer37"":""4"",""answer38"":""3"",""answer39"":""5"",""answer40"":""2"",""answer41"":""4"",""answer42"":""2"",""answer43"":""No change"",""answer44"":""The pandemic, but we\'ve adapted"",""answer45"":""Yes"",""answer46"":""No"",""answer47"":""N/A-I\'m not planning to travel with my dog this Spring"",""answer48"":""Female"",""answer49"":""68"",""answer50"":""2"",""answer51"":""Bachelor\'s degree"",""answer52"":""Retired"",""answer53"":""$50,000 to $74,999"",""answer54"":""Boise"",""answer55"":""Idaho""},{""traits"":""Female aged 60+"",""answer1"":""7"",""answer2"":""Golden Retriever"",""answer3"":""65"",""answer4"":""Sunny"",""answer5"":""Her cheerful personality"",""answer6"":""Sun"",""answer7"":""Before 2020"",""answer8"":""yes"",""answer9"":""10"",""answer10"":""Pure happiness"",""answer11"":""I appreciate the simple joys more"",""answer12"":""5"",""answer13"":""Loyal companion"",""answer14"":""Sunny is always by my side"",""answer15"":""Open doors, took about a month"",""answer16"":""12, on her plush bed in the sunroom"",""answer17"":""yes"",""answer18"":""$1500"",""answer19"":""25%"",""answer20"":""40%"",""answer21"":""$130"",""answer22"":""Spent about the same in March"",""answer23"":""Weekly"",""answer24"":""Dry food"",""answer25"":""I know a lot about the brand"",""answer26"":""5"",""answer27"":""2"",""answer28"":""5"",""answer29"":""4"",""answer30"":""5"",""answer31"":""5"",""answer32"":""1"",""answer33"":""Royal Canin"",""answer34"":""Superb quality, vet recommended"",""answer35"":""5"",""answer36"":""3"",""answer37"":""5"",""answer38"":""4"",""answer39"":""5"",""answer40"":""5"",""answer41"":""2"",""answer42"":""2"",""answer43"":""I interact more"",""answer44"":""Pandemic, we\'re staying home more"",""answer45"":""Not Sure"",""answer46"":""Yes"",""answer47"":""Car"",""answer48"":""Female"",""answer49"":""65"",""answer50"":""1"",""answer51"":""Some college, no degree"",""answer52"":""Employed part-time"",""answer53"":""$25,000 to $49,999"",""answer54"":""Tucson"",""answer55"":""Arizona""},{""traits"":""Female aged 60+"",""answer1"":""9"",""answer2"":""Dachshund"",""answer3"":""14"",""answer4"":""Frankie"",""answer5"":""His long body reminds me of a hotdog"",""answer6"":""Frank"",""answer7"":""In 2020"",""answer8"":""no"",""answer9"":""48"",""answer10"":""Amused by his quirky attitude"",""answer11"":""Laughs are a daily occurrence now"",""answer12"":""5"",""answer13"":""Protector"",""answer14"":""Fearless despite his size"",""answer15"":""Dance, surprisingly one weekend"",""answer16"":""10, in my bed"",""answer17"":""yes"",""answer18"":""$500"",""answer19"":""20%"",""answer20"":""40%"",""answer21"":""$40"",""answer22"":""Spent less in March"",""answer23"":""Monthly"",""answer24"":""Wet food"",""answer25"":""I\'m not very familiar with the brand"",""answer26"":""5"",""answer27"":""3"",""answer28"":""4"",""answer29"":""3"",""answer30"":""4"",""answer31"":""3"",""answer32"":""3"",""answer33"":""Hill\'s Science Diet"",""answer34"":""Recommended by my vet, Frankie\'s health"",""answer35"":""4"",""answer36"":""4"",""answer37"":""4"",""answer38"":""3"",""answer39"":""4"",""answer40"":""4"",""answer41"":""3"",""answer42"":""4"",""answer43"":""I interact more"",""answer44"":""Retirement, I have more time"",""answer45"":""Yes"",""answer46"":""No"",""answer47"":""N/A-I\'m not planning to travel with my dog this Spring"",""answer48"":""Female"",""answer49"":""70"",""answer50"":""1"",""answer51"":""Associate degree"",""answer52"":""Retired"",""answer53"":""Less than $25,000"",""answer54"":""Springfield"",""answer55"":""Missouri""},{""traits"":""Female aged 60+"",""answer1"":""2"",""answer2"":""Poodle"",""answer3"":""10"",""answer4"":""Coco"",""answer5"":""Her fur\'s chocolate color"",""answer6"":""Cokes"",""answer7"":""After 2020"",""answer8"":""yes"",""answer9"":""3"",""answer10"":""Adoration"",""answer11"":""I feel less lonely"",""answer12"":""5"",""answer13"":""Best Friend"",""answer14"":""Coco follows me everywhere"",""answer15"":""Fetch, just a few sessions"",""answer16"":""18, on a designer doggy bed"",""answer17"":""yes"",""answer18"":""$2000"",""answer19"":""10%"",""answer20"":""70%"",""answer21"":""$160"",""answer22"":""Spent more in March"",""answer23"":""Weekly"",""answer24"":""Mix"",""answer25"":""I know a lot about the brand"",""answer26"":""5"",""answer27"":""2"",""answer28"":""5"",""answer29"":""5"",""answer30"":""5"",""answer31"":""3"",""answer32"":""5"",""answer33"":""Orijen"",""answer34"":""Coco\'s improvement, natural ingredients"",""answer35"":""5"",""answer36"":""3"",""answer37"":""5"",""answer38"":""5"",""answer39"":""5"",""answer40"":""3"",""answer41"":""5"",""answer42"":""3"",""answer43"":""I interact more"",""answer44"":""Having moved to a pet-friendly community"",""answer45"":""Yes"",""answer46"":""Yes"",""answer47"":""Car"",""answer48"":""Female"",""answer49"":""61"",""answer50"":""1"",""answer51"":""Graduate degree"",""answer52"":""Self-employed"",""answer53"":""$100,000 to $149,000"",""answer54"":""San Diego"",""answer55"":""California""},{""traits"":""Female aged 60+"",""answer1"":""11"",""answer2"":""Yorkshire Terrier"",""answer3"":""7"",""answer4"":""Pixie"",""answer5"":""Her tiny, fairy-like appearance"",""answer6"":""Pix"",""answer7"":""Before 2020"",""answer8"":""no"",""answer9"":""12"",""answer10"":""A bit anxious, she was so small"",""answer11"":""I enjoy everyday moments more"",""answer12"":""5"",""answer13"":""Family Member"",""answer14"":""She\'s been with me through thick and thin"",""answer15"":""Spin, just took two weeks"",""answer16"":""13, her cushioned crate"",""answer17"":""yes"",""answer18"":""$700"",""answer19"":""15%"",""answer20"":""45%"",""answer21"":""$60"",""answer22"":""Spent about the same in March"",""answer23"":""Bi-weekly"",""answer24"":""Dry food"",""answer25"":""I know just the basics about the brand"",""answer26"":""4"",""answer27"":""3"",""answer28"":""4"",""answer29"":""5"",""answer30"":""5"",""answer31"":""1"",""answer32"":""4"",""answer33"":""Iams"",""answer34"":""My Pixie\'s health, cost-effective"",""answer35"":""4"",""answer36"":""5"",""answer37"":""4"",""answer38"":""4"",""answer39"":""5"",""answer40"":""1"",""answer41"":""4"",""answer42"":""5"",""answer43"":""I interact more"",""answer44"":""The pandemic has us spending more time indoors"",""answer45"":""Not Sure"",""answer46"":""No"",""answer47"":""N/A-I\'m not planning to travel with my dog this Spring"",""answer48"":""Female"",""answer49"":""72"",""answer50"":""2"",""answer51"":""High school diploma or equivalent"",""answer52"":""Retired"",""answer53"":""Less than $25,000"",""answer54"":""Macon"",""answer55"":""Georgia""},{""traits"":""Female aged 60+"",""answer1"":""4"",""answer2"":""Border Collie"",""answer3"":""45"",""answer4"":""Shep"",""answer5"":""Traditional name for a sheepdog"",""answer6"":""Sheppy"",""answer7"":""Before 2020"",""answer8"":""yes"",""answer9"":""5"",""answer10"":""I felt protective"",""answer11"":""Daily physical activity is a must"",""answer12"":""5"",""answer13"":""Loyal companion"",""answer14"":""Always ready to work and play"",""answer15"":""We\'ve mastered herding techniques, took months"",""answer16"":""10, in the utility room on his mat"",""answer17"":""yes"",""answer18"":""$1300"",""answer19"":""10%"",""answer20"":""55%"",""answer21"":""$110"",""answer22"":""Spent about the same in March"",""answer23"":""Weekly"",""answer24"":""Dry food"",""answer25"":""I know a lot about the brand"",""answer26"":""5"",""answer27"":""4"",""answer28"":""5"",""answer29"":""3"",""answer30"":""5"",""answer31"":""3"",""answer32"":""2"",""answer33"":""Acana"",""answer34"":""High-quality, breed-specific formula"",""answer35"":""5"",""answer36"":""4"",""answer37"":""5"",""answer38"":""4"",""answer39"":""5"",""answer40"":""3"",""answer41"":""2"",""answer42"":""4"",""answer43"":""No change"",""answer44"":""Seasonal allergies, we adjust our time outside"",""answer45"":""No"",""answer46"":""No"",""answer47"":""N/A-I\'m not planning to travel with my dog this Spring"",""answer48"":""Female"",""answer49"":""67"",""answer50"":""1"",""answer51"":""Bachelor\'s degree"",""answer52"":""Employed part-time"",""answer53"":""$25,000 to $49,999"",""answer54"":""Topeka"",""answer55"":""Kansas""},{""traits"":""Female aged 60+"",""answer1"":""10"",""answer2"":""Chihuahua"",""answer3"":""5"",""answer4"":""Tiny"",""answer5"":""Her petite size"",""answer6"":""T"",""answer7"":""After 2020"",""answer8"":""yes"",""answer9"":""24"",""answer10"":""Amused by her sassiness"",""answer11"":""It\'s important to love fiercely"",""answer12"":""5"",""answer13"":""Best Friend"",""answer14"":""Tiny is always by my side, my constant tiny shadow"",""answer15"":""Sit pretty, she picked it up in days"",""answer16"":""14, she curls up in the bed under the window"",""answer17"":""yes"",""answer18"":""$550"",""answer19"":""40%"",""answer20"":""30%"",""answer21"":""$50"",""answer22"":""Spent less in March"",""answer23"":""Bi-weekly"",""answer24"":""Mix"",""answer25"":""I know just the basics about the brand"",""answer26"":""5"",""answer27"":""5"",""answer28"":""5"",""answer29"":""2"",""answer30"":""5"",""answer31"":""2"",""answer32"":""1"",""answer33"":""Cesar"",""answer34"":""Tiny loves it, it\'s affordable, easy to store"",""answer35"":""5"",""answer36"":""5"",""answer37"":""4"",""answer38"":""2"",""answer39"":""5"",""answer40"":""2"",""answer41"":""1"",""answer42"":""5"",""answer43"":""No change"",""answer44"":""The seasons changing is the biggest influence"",""answer45"":""Yes"",""answer46"":""Yes"",""answer47"":""Car"",""answer48"":""Female"",""answer49"":""69"",""answer50"":""1"",""answer51"":""Some college, no degree"",""answer52"":""Retired"",""answer53"":""$50,000 to $74,999"",""answer54"":""El Paso"",""answer55"":""Texas""},{""traits"":""Female aged 60+"",""answer1"":""6"",""answer2"":""Bulldog"",""answer3"":""50"",""answer4"":""Winston"",""answer5"":""His stately, British-like demeanor"",""answer6"":""Win"",""answer7"":""Before 2020"",""answer8"":""no"",""answer9"":""3"",""answer10"":""Awe, he\'s got so much character"",""answer11"":""I\'ve become more patient"",""answer12"":""5"",""answer13"":""Family Member"",""answer14"":""He\'s more like a child to me than a pet"",""answer15"":""Speak on command, a stubborn month or so"",""answer16"":""12, sprawled across the hallway rug"",""answer17"":""yes"",""answer18"":""$1100"",""answer19"":""15%"",""answer20"":""45%"",""answer21"":""$90"",""answer22"":""Spent about the same in March"",""answer23"":""Monthly"",""answer24"":""Dry food"",""answer25"":""I know a lot about the brand"",""answer26"":""4"",""answer27"":""2"",""answer28"":""4"",""answer29"":""5"",""answer30"":""5"",""answer31"":""3"",""answer32"":""3"",""answer33"":""Nutro"",""answer34"":""Non-GMO ingredients, Winston\'s preference"",""answer35"":""4"",""answer36"":""3"",""answer37"":""4"",""answer38"":""5"",""answer39"":""5"",""answer40"":""3"",""answer41"":""3"",""answer42"":""3"",""answer43"":""I interact less"",""answer44"":""Winston\'s aging, he is more independent"",""answer45"":""Not Sure"",""answer46"":""No"",""answer47"":""N/A-I\'m not planning to travel with my dog this Spring"",""answer48"":""Female"",""answer49"":""65"",""answer50"":""2"",""answer51"":""Graduate degree"",""answer52"":""Retired"",""answer53"":""Prefer not to say"",""answer54"":""Charleston"",""answer55"":""South Carolina""},{""traits"":""Female aged 60+"",""answer1"":""8"",""answer2"":""Shih Tzu"",""answer3"":""12"",""answer4"":""Gizmo"",""answer5"":""His playful and curious nature"",""answer6"":""Gizzy"",""answer7"":""In 2020"",""answer8"":""yes"",""answer9"":""60"",""answer10"":""Instant love, he\'s such a fluffball"",""answer11"":""I\'m more outgoing, he needs lots of socializing"",""answer12"":""5"",""answer13"":""Best Friend"",""answer14"":""We have an unbreakable bond"",""answer15"":""Roll over, a week\'s full of treats and praise"",""answer16"":""15, on his favorite armchair"",""answer17"":""yes"",""answer18"":""$950"",""answer19"":""10%"",""answer20"":""55%"",""answer21"":""$80"",""answer22"":""Spent about the same in March"",""answer23"":""Monthly"",""answer24"":""Dry food"",""answer25"":""I know just the basics about the brand"",""answer26"":""5"",""answer27"":""4"",""answer28"":""5"",""answer29"":""3"",""answer30"":""5"",""answer31"":""1"",""answer32"":""4"",""answer33"":""Science Diet"",""answer34"":""Gizzy likes it, vet approved, easy to find"",""answer35"":""4"",""answer36"":""4"",""answer37"":""5"",""answer38"":""3"",""answer39"":""5"",""answer40"":""1"",""answer41"":""4"",""answer42"":""4"",""answer43"":""No change"",""answer44"":""Seasonal weather, we keep our routine"",""answer45"":""Yes"",""answer46"":""Not Sure"",""answer47"":""N/A-I\'m not planning to travel with my dog this Spring"",""answer48"":""Female"",""answer49"":""64"",""answer50"":""1"",""answer51"":""Bachelor\'s degree"",""answer52"":""Employed full-time"",""answer53"":""$50,000 to $74,999"",""answer54"":""Reno"",""answer55"":""Nevada""},{""traits"":""Female aged 60+"",""answer1"":""1"",""answer2"":""Australian Shepherd"",""answer3"":""30"",""answer4"":""Blue"",""answer5"":""His striking blue eyes"",""answer6"":""Boo"",""answer7"":""After 2020"",""answer8"":""no"",""answer9"":""2"",""answer10"":""Thrilled, he was full of energy"",""answer11"":""I\'ve got a new sense of purpose"",""answer12"":""5"",""answer13"":""Loyal companion"",""answer14"":""He\'s loyal and always there for support"",""answer15"":""Fetch with a frisbee, about two weeks"",""answer16"":""14, on a runner in the hall"",""answer17"":""yes"",""answer18"":""$1000"",""answer19"":""5%"",""answer20"":""65%"",""answer21"":""$85"",""answer22"":""Spent more in March"",""answer23"":""Weekly"",""answer24"":""Dry food"",""answer25"":""I know a lot about the brand"",""answer26"":""5"",""answer27"":""3"",""answer28"":""5"",""answer29"":""4"",""answer30"":""5"",""answer31"":""4"",""answer32"":""1"",""answer33"":""Taste of the Wild"",""answer34"":""Grain-free, Blue\'s coat looks amazing"",""answer35"":""5"",""answer36"":""4"",""answer37"":""5"",""answer38"":""4"",""answer39"":""5"",""answer40', 'name': 'Answers'}})


In [ ]:
b = AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{""answer"":[{""traits"":""Female aged 60+"",""answer1"":""7"",""answer2"":""Beagle"",""answer3"":""28"",""answer4"":""Bailey"",""answer5"":""It was the name she had been given at the shelter, and it suited her well."",""answer6"":""Bails"",""answer7"":""Before 2020"",""answer8"":""yes"",""answer9"":""18"",""answer10"":""I was overwhelmed with joy and a bit anxious about whether I could provide a good home."",""answer11"":""She\'s brought so much joy and a daily routine that keeps me active."",""answer12"":""5"",""answer13"":""Best Friend"",""answer14"":""Bailey is always there for me, through the good and bad times."",""answer15"":""Shake hands, about a week."",""answer16"":""14, she has a cozy bed in my bedroom."",""answer17"":""yes"",""answer18"":""$1200"",""answer19"":""25%"",""answer20"":""50%"",""answer21"":""$100"",""answer22"":""Spent about the same in March"",""answer23"":""Monthly"",""answer24"":""Dry food"",""answer25"":""I know just the basics about the brand"",""answer26"":""5"",""answer27"":""4"",""answer28"":""5"",""answer29"":""3"",""answer30"":""5"",""answer31"":""3"",""answer32"":""2"",""answer33"":""Purina"",""answer34"":""Good value, she enjoys it, widely available."",""answer35"":""5"",""answer36"":""4"",""answer37"":""5"",""answer38"":""4"",""answer39"":""5"",""answer40"":""3"",""answer41"":""2"",""answer42"":""2"",""answer43"":""No change"",""answer44"":""None in particular"",""answer45"":""No"",""answer46"":""No"",""answer47"":""N/A-I\'m not planning to travel with my dog this Spring"",""answer48"":""Female"",""answer49"":""65"",""answer50"":""1"",""answer51"":""Graduate degree"",""answer52"":""Retired"",""answer53"":""$50,000 to $74,999"",""answer54"":""Sarasota"",""answer55"":""Florida""},{""traits"":""Female aged 60+"",""answer1"":""10"",""answer2"":""Labrador Retriever"",""answer3"":""72"",""answer4"":""Max"",""answer5"":""Max just seemed like a strong and friendly name, which matched his personality."",""answer6"":""Maxie"",""answer7"":""After 2020"",""answer8"":""no"",""answer9"":""8"",""answer10"":""Instant love, like we were meant to find each other."",""answer11"":""He reminds me to appreciate the little moments."",""answer12"":""5"",""answer13"":""Family Member"",""answer14"":""He\'s not just a pet; he\'s a part of the family."",""answer15"":""\'Leave it\' command, took around two weeks."",""answer16"":""10, Max has his own bed in the living room."",""answer17"":""yes"",""answer18"":""$1800"",""answer19"":""30%"",""answer20"":""40%"",""answer21"":""$150"",""answer22"":""Spent more in March"",""answer23"":""Weekly"",""answer24"":""Mix"",""answer25"":""I know a lot about the brand"",""answer26"":""5"",""answer27"":""3"",""answer28"":""5"",""answer29"":""4"",""answer30"":""5"",""answer31"":""4"",""answer32"":""4"",""answer33"":""Royal Canin"",""answer34"":""High-quality ingredients, recommended by our vet, Max\'s preference."",""answer35"":""5"",""answer36"":""3"",""answer37"":""5"",""answer38"":""4"",""answer39"":""5"",""answer40"":""4"",""answer41"":""4"",""answer42"":""3"",""answer43"":""I interact more"",""answer44"":""The ongoing pandemic has given us more time together."",""answer45"":""Yes"",""answer46"":""No"",""answer47"":""Car"",""answer48"":""Female"",""answer49"":""68"",""answer50"":""2"",""answer51"":""Bachelor\'s degree"",""answer52"":""Employed part-time"",""answer53"":""$50,000 to $74,999"",""answer54"":""Albuquerque"",""answer55"":""New Mexico""},{""traits"":""Female aged 60+"",""answer1"":""3"",""answer2"":""Golden Retriever"",""answer3"":""65"",""answer4"":""Sunny"",""answer5"":""She has a bright golden coat and a sunny disposition!"",""answer6"":""Sunshine"",""answer7"":""After 2020"",""answer8"":""yes"",""answer9"":""10"",""answer10"":""It felt like a dream, she was so sweet and gentle."",""answer11"":""Sunny has made me more compassionate and patient."",""answer12"":""5"",""answer13"":""Loyal Companion"",""answer14"":""Through thick and thin, she\'s always by my side, loyal as they come."",""answer15"":""\'Stay,\' about three days with consistent practice."",""answer16"":""12, a plush bed next to mine."",""answer17"":""yes"",""answer18"":""$900"",""answer19"":""20%"",""answer20"":""60%"",""answer21"":""$75"",""answer22"":""Spent less in March"",""answer23"":""Monthly"",""answer24"":""Dry food"",""answer25"":""I know just the basics about the brand"",""answer26"":""5"",""answer27"":""3"",""answer28"":""5"",""answer29"":""3"",""answer30"":""5"",""answer31"":""3"",""answer32"":""1"",""answer33"":""Hill\'s Science Diet"",""answer34"":""She loves it, seems to keep her healthy."",""answer35"":""5"",""answer36"":""4"",""answer37"":""5"",""answer38"":""4"",""answer39"":""5"",""answer40"":""3"",""answer41"":""2"",""answer42"":""2"",""answer43"":""No change"",""answer44"":""None in particular"",""answer45"":""Not Sure"",""answer46"":""Yes"",""answer47"":""Car"",""answer48"":""Female"",""answer49"":""72"",""answer50"":""1"",""answer51"":""Associate degree"",""answer52"":""Retired"",""answer53"":""Less than $25,000"",""answer54"":""Eugene"",""answer55"":""Oregon""},{""traits"":""Female aged 60+"",""answer1"":""5"",""answer2"":""Poodle"",""answer3"":""15"",""answer4"":""Coco"",""answer5"":""After Chanel, because Coco has such a stylish and sophisticated aura."",""answer6"":""Cokes"",""answer7"":""Before 2020"",""answer8"":""no"",""answer9"":""24"",""answer10"":""Nervous but also excited, she was adorably shy."",""answer11"":""In her gentle way, she has taught me to be calmer."",""answer12"":""5"",""answer13"":""Protector"",""answer14"":""Despite her size, she\'s always alert and makes me feel safe."",""answer15"":""\'Dance\' on hind legs, took her a couple of days."",""answer16"":""16, Coco sleeps in a small bed beside my desk."",""answer17"":""yes"",""answer18"":""$700"",""answer19"":""10%"",""answer20"":""70%"",""answer21"":""$60"",""answer22"":""I\'m not sure"",""answer23"":""Bi-weekly"",""answer24"":""Wet food"",""answer25"":""I know a lot about the brand"",""answer26"":""4"",""answer27"":""2"",""answer28"":""4"",""answer29"":""5"",""answer30"":""5"",""answer31"":""5"",""answer32"":""3"",""answer33"":""Blue Buffalo"",""answer34"":""Quality ingredients, variety of flavors, and Coco’s contentment."",""answer35"":""5"",""answer36"":""3"",""answer37"":""5"",""answer38"":""5"",""answer39"":""5"",""answer40"":""5"",""answer41"":""3"",""answer42"":""4"",""answer43"":""No change"",""answer44"":""None in particular"",""answer45"":""No"",""answer46"":""Not Sure"",""answer47"":""Other"",""answer48"":""Female"",""answer49"":""63"",""answer50"":""1"",""answer51"":""Graduate degree"",""answer52"":""Self-employed"",""answer53"":""$100,000 to $149,000"",""answer54"":""Austin"",""answer55"":""Texas""},{""traits"":""Female aged 60+"",""answer1"":""8"",""answer2"":""Dachshund"",""answer3"":""22"",""answer4"":""Frankie"",""answer5"":""He reminded me of old Frankie Sinatra because of his smooth charm."",""answer6"":""Frank"",""answer7"":""Before 2020"",""answer8"":""yes"",""answer9"":""36"",""answer10"":""I couldn\'t help but laugh; he had such a funny, energetic vibe."",""answer11"":""Frankie demonstrates every day that age is just a number."",""answer12"":""5"",""answer13"":""Family Member"",""answer14"":""He\'s been a constant in my life, just as any family member would."",""answer15"":""The \'beg\' trick. It took almost a month of sporadic practice."",""answer16"":""12, he\'s got a little bed in the corner of the living room."",""answer17"":""yes"",""answer18"":""$1100"",""answer19"":""15%"",""answer20"":""55%"",""answer21"":""$90"",""answer22"":""Spent about the same in March"",""answer23"":""Bi-weekly"",""answer24"":""Mix"",""answer25"":""I know a lot about the brand"",""answer26"":""4"",""answer27"":""3"",""answer28"":""4"",""answer29"":""4"",""answer30"":""5"",""answer31"":""2"",""answer32"":""2"",""answer33"":""Iams"",""answer34"":""Frankie likes it, I trust the brand, and it\'s a decent price."",""answer35"":""4"",""answer36"":""3"",""answer37"":""4"",""answer38"":""4"",""answer39"":""5"",""answer40"":""2"",""answer41"":""2"",""answer42"":""3"",""answer43"":""I interact less"",""answer44"":""I\'ve had to limit our outdoor interactions due to my recent knee surgery."",""answer45"":""Yes"",""answer46"":""Yes"",""answer47"":""Car"",""answer48"":""Female"",""answer49"":""71"",""answer50"":""2"",""answer51"":""Bachelor\'s degree"",""answer52"":""Retired"",""answer53"":""$50,000 to $74,999"",""answer54"":""Colorado Springs"",""answer55"":""Colorado""}]}', 'name': 'Answers'}})


In [ ]:
b

In [ ]:
tt = b.additional_kwargs['function_call']['arguments']

In [ ]:
tt

In [ ]:
t = ','.join(a.additional_kwargs['function_call']['arguments'].split(',')[:-1])+'}]}'

In [ ]:
t  =t.replace('""', '"')

In [ ]:
t

In [ ]:
json.loads(t)

In [ ]:
b.additional_kwargs['function_call']['arguments'] = b.additional_kwargs['function_call']['arguments'].replace('""', '"')

In [ ]:
json.loads(b.additional_kwargs['function_call']['arguments'])['answer']

In [ ]:
import re
data = json.loads(re.sub(r"(\w+):", r'"\1":', r"b.additional_kwargs['function_call']['arguments']))

In [ ]:
print(x)

In [ ]:
i='{""a"":""1"", ""b"":""2"", ""c""}'
i = ','.join(i.split(',')[:-1])+'}'
print(i)
i = i.replace('""', '"')
print(i)
json.loads(i)